# Typical Workflow

In [1]:
# 1) loading data, 2) training, 3) validation, 4) testing
import tensorflow as tf
import keras
from keras import layers

In [3]:
# create model
inputs = keras.Input(shape=(784,), name="digits")
x = layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = layers.Dense(10, activation="softmax", name="predictions")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [8]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
# Preprocess the data (flatten and normalise)
x_train = x_train.reshape(60000, 784).astype("float32") / 255
x_test = x_test.reshape(10000, 784).astype("float32") / 255

y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

# Reserve 10,000 samples for validation
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

In [10]:
model.compile(
    optimizer=keras.optimizers.RMSprop(),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

In [11]:
print("Fit model on training data")
history = model.fit(
    x_train,
    y_train,
    batch_size=64,
    epochs=2,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    validation_data=(x_val, y_val),
)
# instead of using validation_data like this we can juse use the train/val split argument
# history = model.fit(
#    x_train, 
#    y_train, 
#    batch_size=64, 
#    validation_split=0.2, # would take 20% for validation
#    epochs=2)

Fit model on training data
Epoch 1/2
782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 840us/step - loss: 0.5593 - sparse_categorical_accuracy: 0.8420 - val_loss: 0.2031 - val_sparse_categorical_accuracy: 0.9430
Epoch 2/2
782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 675us/step - loss: 0.1766 - sparse_categorical_accuracy: 0.9485 - val_loss: 0.1514 - val_sparse_categorical_accuracy: 0.9566


In [12]:
history.history

{'loss': [0.3421056270599365, 0.16146428883075714],
 'sparse_categorical_accuracy': [0.9017199873924255, 0.9519400000572205],
 'val_loss': [0.20314480364322662, 0.15140461921691895],
 'val_sparse_categorical_accuracy': [0.9430000185966492, 0.95660001039505]}

In [16]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(x_test, y_test, batch_size=128)
print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for 3 samples")
predictions = model.predict(x_test[:3])
print("predictions shape:", predictions.shape)

# these are the softmax probabilities, to get the classes we just apply argmax
predicted_classes = tf.argmax(predictions, axis=1)
print("Predicted class labels:", predicted_classes)

Evaluate on test data
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 491us/step - loss: 0.1798 - sparse_categorical_accuracy: 0.9469
test loss, test acc: [0.1554391086101532, 0.9535999894142151]
Generate predictions for 3 samples
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
predictions shape: (3, 10)
Predicted class labels: tf.Tensor([7 2 1], shape=(3,), dtype=int64)


# compile method

In [18]:
# compile method needs an optimizer, a loss function and optionally one or more metrics as a list
model.compile(
    optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
# most can also just be implemented through strings
model.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)


In [19]:
# for convenience
def get_uncompiled_model():
    inputs = keras.Input(shape=(784,), name="digits")
    x = layers.Dense(64, activation="relu", name="dense_1")(inputs)
    x = layers.Dense(64, activation="relu", name="dense_2")(x)
    outputs = layers.Dense(10, activation="softmax", name="predictions")(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


def get_compiled_model():
    model = get_uncompiled_model()
    model.compile(
        optimizer="rmsprop",
        loss="sparse_categorical_crossentropy",
        metrics=["sparse_categorical_accuracy"],
    )
    return model


## Custom loss

In [20]:
# custom loss functions can be implemented. They require 2 parameters, y_true, y_pred
def custom_mean_squared_error(y_true, y_pred):
    return tf.math.reduce_mean(tf.square(y_true - y_pred), axis=-1)


model = get_uncompiled_model()
model.compile(optimizer=keras.optimizers.Adam(), loss=custom_mean_squared_error)

# We need to one-hot encode the labels to use MSE
y_train_one_hot = tf.one_hot(y_train, depth=10)
model.fit(x_train, y_train_one_hot, batch_size=64, epochs=1)


782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 628us/step - loss: 0.0284


In [ ]:
# for more flexibility you can create an own loss class. 
@keras.saving.register_keras_serializable() # this makes sure tf can track the class
class CustomMSE(keras.losses.Loss):
    def __init__(self, regularization_factor=0.1, name="custom_mse"):
        super().__init__(name=name)
        self.regularization_factor = regularization_factor

    def call(self, y_true, y_pred):
        mse = tf.math.reduce_mean(tf.square(y_true - y_pred), axis=-1)
        reg = tf.math.reduce_mean(tf.square(0.5 - y_pred), axis=-1)
        return mse + reg * self.regularization_factor

    def get_config(self):
        return {
            "regularization_factor": self.regularization_factor,
            "name": self.name,
        }


model = get_uncompiled_model()
model.compile(optimizer=keras.optimizers.Adam(), loss=CustomMSE())

y_train_one_hot = tf.one_hot(y_train, depth=10)
model.fit(x_train, y_train_one_hot, batch_size=64, epochs=1)


# Custom Metric

In [22]:
@keras.saving.register_keras_serializable()
class CategoricalTruePositives(keras.metrics.Metric):
    def __init__(self, name="categorical_true_positives", **kwargs):
        super().__init__(name=name, **kwargs)
        self.true_positives = self.add_weight(name="ctp", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.reshape(tf.argmax(y_pred, axis=1), shape=(-1, 1))
        values = tf.cast(y_true, "int32") == tf.cast(y_pred, "int32")
        values = tf.cast(values, "float32")
        if sample_weight is not None:
            sample_weight = tf.cast(sample_weight, "float32")
            values = tf.multiply(values, sample_weight)
        self.true_positives.assign_add(tf.reduce_sum(values))

    def result(self):
        return self.true_positives

    def reset_state(self):
        # The state of the metric will be reset at the start of each epoch.
        self.true_positives.assign(0.0)


model = get_uncompiled_model()
model.compile(
    optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[CategoricalTruePositives()],
)
model.fit(x_train, y_train, batch_size=64, epochs=3)


Epoch 1/3
782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 592us/step - categorical_true_positives: 180315.5469 - loss: 0.5819
Epoch 2/3
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 595us/step - categorical_true_positives: 182180.5000 - loss: 0.1706
Epoch 3/3
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 594us/step - categorical_true_positives: 183681.3438 - loss: 0.1152


In [24]:
# majority of loss functions are implemented or can be implemented using y_true and y_pred.
# if that's not the case like for regularization loss, then we can use add_loss to add loss to the main loss or 
# instead of the loss (compile model without loss function)
@keras.saving.register_keras_serializable()
class ActivityRegularizationLayer(layers.Layer):
    def call(self, inputs):
        self.add_loss(tf.reduce_sum(inputs) * 0.1)
        return inputs  # Pass-through layer.


inputs = keras.Input(shape=(784,), name="digits")
x = layers.Dense(64, activation="relu", name="dense_1")(inputs)

# Insert activity regularization as a layer
x = ActivityRegularizationLayer()(x)

x = layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = layers.Dense(10, name="predictions")(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(
    optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
)

# The displayed loss will be much higher than before
# due to the regularization component.
model.fit(x_train, y_train, batch_size=64, epochs=1)

782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 688us/step - loss: 3.2492
